<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/VAA_aggresive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
pip install yfinance

In [135]:
pip install pandas_market_calendars

In [136]:
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()
import pandas as pd
import exchange_calendars as ecals
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.max_columns', None)

def get_price_data(x, start_date, end_date):
  df_price = pd.DataFrame(columns=x)
  for ticker in x:
    df_price[ticker] = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)['Adj Close']
  return df_price.dropna(axis=0)

# 1년 전 개장일 구하기
us_market_code = "XNYS"
us_calendar = ecals.get_calendar(us_market_code)

# 각 날짜 변수에 저장
today = datetime.today().date()
today_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=0)).date())
month_01_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=30)).date())
month_03_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=91)).date())
month_06_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=182)).date())
month_12_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=365)).date())

# 전략에 사용할 ETF 명 리스트에 담기
VAA_agg = ['SPY', 'EFA', 'EEM', 'AGG', 'LQD', 'IEF', 'SHY']

df_VAA_agg = get_price_data(VAA_agg, month_12_ago_workday, today).sort_index(ascending=False)
display(df_VAA_agg)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,SPY,EFA,EEM,AGG,LQD,IEF,SHY
Date,,,,,,,
2023-08-11,445.6500,72.1300,39.5600,96.0700,105.0500,94.1300,80.9700
2023-08-10,445.9100,72.6200,40.1500,96.3700,105.4100,94.6300,81.0500
2023-08-09,445.7500,72.3100,40.0900,97.0000,106.1600,95.3600,81.0900
2023-08-08,448.7500,72.3300,39.9900,96.9300,106.0600,95.3000,81.1400
2023-08-07,450.7100,72.7700,40.4800,96.7000,105.6400,94.8200,81.1000
...,...,...,...,...,...,...,...
2022-08-18,421.0150,64.1521,39.5330,99.7845,108.6893,100.7491,80.5913
2022-08-17,419.7948,64.3673,39.7578,99.5708,108.4102,100.5542,80.4742
2022-08-16,422.7958,64.8857,39.9240,100.1634,109.4399,101.2364,80.5425


In [137]:
ago_name = ['today', 'month_01_ago', 'month_03_ago', 'month_06_ago', 'month_12_ago']
ago_date = [today_workday.date(), \
            month_01_ago_workday.date(), \
            month_03_ago_workday.date(), \
            month_06_ago_workday.date(), \
            month_12_ago_workday.date()]

ago = [f'{a}/{d}' for a, d in zip(ago_name, ago_date)]

etf_data = {}

for i in range(7):
  etf_data[VAA_agg[i]] = [df_VAA_agg.loc[today_workday, VAA_agg[i]], \
                          df_VAA_agg.loc[month_01_ago_workday, VAA_agg[i]], \
                          df_VAA_agg.loc[month_03_ago_workday, VAA_agg[i]], \
                          df_VAA_agg.loc[month_06_ago_workday, VAA_agg[i]], \
                          df_VAA_agg.loc[month_12_ago_workday, VAA_agg[i]]]

## DataFrame 만들기: 1-3-6-12 개월 etf 가격
df_etf_price = pd.DataFrame(etf_data, columns=VAA_agg, index=ago)
str_etf_price = "### ETF 가격 현재-1-3-6-12 개월"
display(Markdown(str_etf_price))
display(df_etf_price)

# csv 경로 설정
output_path =  '/content/drive/MyDrive/colab/'

# 오늘 날짜를 가져와서 파일명에 추가
today_date = datetime.now().strftime('%y%m%d')
filename = f'{today_date} VAA_agg.xlsx'

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'w') as f:
  f.write(f'{str_etf_price}\n')
  df_etf_price.to_csv(f, index=True)

### ETF 가격 현재-1-3-6-12 개월

,SPY,EFA,EEM,AGG,LQD,IEF,SHY
today/2023-08-11,445.6500,72.1300,39.5600,96.0700,105.0500,94.1300,80.9700
month_01_ago/2023-07-13,449.5600,74.2800,41.2400,98.1430,107.9195,96.7648,81.1673
month_03_ago/2023-05-12,410.0668,71.8234,38.2503,98.6036,107.2531,98.7461,81.5499
month_06_ago/2023-02-10,404.9843,69.3979,39.8776,96.9181,105.6354,95.6377,80.1027
month_12_ago/2022-08-12,420.2376,65.2965,40.1194,100.1829,109.9403,101.0317,80.5425


In [138]:
etf_momentum_score = []
etf_ror_data = []

## DataFrame 만들기: 수익률
for etf_name in VAA_agg:
  etf_today_price = df_etf_price.loc[ago[0], etf_name]
  etf_momentum_score.append((12 * (etf_today_price/df_etf_price.loc[ago[1], etf_name] - 1) * 100) + \
                            (4 * (etf_today_price/df_etf_price.loc[ago[2], etf_name] - 1) * 100) + \
                            (2 * (etf_today_price/df_etf_price.loc[ago[3], etf_name] - 1) * 100) + \
                            (1 * (etf_today_price/df_etf_price.loc[ago[4], etf_name] - 1) * 100))

  etf_ror_data.append([(etf_today_price/df_etf_price.loc[ago[1], etf_name] - 1) * 100, \
                       (etf_today_price/df_etf_price.loc[ago[2], etf_name] - 1) * 100, \
                       (etf_today_price/df_etf_price.loc[ago[3], etf_name] - 1) * 100, \
                       (etf_today_price/df_etf_price.loc[ago[4], etf_name] - 1) * 100])

df_etf_ror = pd.DataFrame(etf_ror_data, columns=ago[1:], index=VAA_agg).transpose()

str_etf_ror = "### ETF 1-3-6-12 개월 수익률"
display(Markdown(str_etf_ror))
display(df_etf_ror)

# csv에 제목 및 DataFrame 추가
with open(filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror}\n')
  df_etf_ror.to_csv(f, index=True)
  # df_etf_ror.to_csv(filename, mode='a', header=True, index=True)


## DataFrame 만들기: 수익률 가중치
df_etf_ror_weight = df_etf_ror
df_etf_ror_weight = df_etf_ror_weight.apply(lambda row: row * 12 if row.name == ago[1] else row, axis=1)
df_etf_ror_weight = df_etf_ror_weight.apply(lambda row: row * 4 if row.name == ago[2] else row, axis=1)
df_etf_ror_weight = df_etf_ror_weight.apply(lambda row: row * 2 if row.name == ago[3] else row, axis=1)

str_etf_ror_weight = "### ETF 1-3-6-12 개월 가중치 적용"
display(Markdown(str_etf_ror_weight))
display(df_etf_ror_weight)

# csv에 제목 및 DataFrame 추가
with open(filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror_weight}\n')
  df_etf_ror_weight.to_csv(f, index=True)

## DataFrame 만들기: 모멘텀
df_etf_momentum_score = pd.DataFrame(etf_momentum_score, columns=['momentum_score'], index=VAA_agg).transpose()

str_etf_momentum_score = "### ETF 별 12개월 Momentum Score"
display(Markdown(str_etf_momentum_score))
display(df_etf_momentum_score)

# csv에 제목 및 DataFrame 추가
with open(filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_momentum_score}\n')
  df_etf_momentum_score.to_csv(f, index=True)

### ETF 1-3-6-12 개월 수익률

,SPY,EFA,EEM,AGG,LQD,IEF,SHY
month_01_ago/2023-07-13,-0.8697,-2.8945,-4.0737,-2.1122,-2.6589,-2.7229,-0.2431
month_03_ago/2023-05-12,8.6774,0.4269,3.4239,-2.5694,-2.0541,-4.6747,-0.7111
month_06_ago/2023-02-10,10.0413,3.9369,-0.7964,-0.8750,-0.5541,-1.5765,1.0827
month_12_ago/2022-08-12,6.0471,10.4654,-1.3944,-4.1054,-4.4482,-6.8312,0.5308


### ETF 1-3-6-12 개월 가중치 적용

,SPY,EFA,EEM,AGG,LQD,IEF,SHY
month_01_ago/2023-07-13,-10.4369,-34.7335,-48.8846,-25.3469,-31.9070,-32.6746,-2.9172
month_03_ago/2023-05-12,34.7097,1.7074,13.6957,-10.2777,-8.2163,-18.6989,-2.8442
month_06_ago/2023-02-10,20.0826,7.8738,-1.5928,-1.7500,-1.1083,-3.1529,2.1654
month_12_ago/2022-08-12,6.0471,10.4654,-1.3944,-4.1054,-4.4482,-6.8312,0.5308


### ETF 별 12개월 Momentum Score

,SPY,EFA,EEM,AGG,LQD,IEF,SHY
momentum_score,50.4026,-14.6869,-38.1760,-41.4801,-45.6798,-61.3577,-3.0652


In [141]:
# 0 이하인 값이 있는지 확인 후 어떤 자산 매수할 지 산출
has_negative_values = any(score <= 0 for score in etf_momentum_score[:4])
str_VAA_agg = ""
etf_VAA_agg = ""

if has_negative_values:
  str_VAA_agg = "## 공격형 자산 중 0 이하 발생! 모멘텀 스코어 가장 높은 안전 자산 매수 필요\n"
  etf_VAA_agg = "## 추천 안전 자산: " + VAA_agg[etf_momentum_score.index(max(etf_momentum_score[4:]))]
else:
  str_VAA_agg = "## 공격형 자산 모두 0 이상! 모멘텀 스코어 가장 높은 공격 자산 매수 필요\n"
  etf_VAA_agg = "## 추천 공격 자산: " + VAA_agg[etf_momentum_score.index(max(etf_momentum_score[:4]))]

display(Markdown(str_VAA_agg))
display(Markdown(etf_VAA_agg))

# csv에 자산 투자 조언 추가
with open(filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_VAA_agg}')
  f.write(f'{etf_VAA_agg}')


## 공격형 자산 중 0 이하 발생! 모멘텀 스코어 가장 높은 안전 자산 매수 필요


## 추천 안전 자산: SHY